# Capstone

### This notebook will be used for the coursera capstone project

In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

from geopy.geocoders import Nominatim

import folium

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [3]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


<b>purpose of this section is to retrieve the boroughs and the neighbourhoods in Toronto</b>
<ul><li> step 1 : scrape wikipediapage to get a list of all neighbourhoods
<li> step 2 : extract the correct table using BeautifulSoup / clease irrelevant data until only the rows containing neighbourhoods is left
<li> step 3 : assign the neighbourhoods & associated boroughs from the wikipedia page to a dataframe 
<li> step 4 : use geocoders to find the latitude and longitude of each neighbourhood
    <br> ! only those neighourhoods that return a geolocation will be kept, the rest is discarded
</ul>

In [4]:
url = 'https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Toronto'
raw_page = requests.get(url).text

In [5]:
soup = BeautifulSoup(raw_page, 'html.parser')

In [6]:
#table1 = soup.findAll("tbody", {"class": "wikitable sortable jquery-tablesorter"})
#table1 = soup.findAll('table', class_="wikitable sortable jquery-tablesorter")
table1 = soup.find_all('table',class_="wikitable")
#tbody1 = soup.findAll('tbody',table1)

In [7]:
raw_neighbourhoods_table = table1[1].find_all("tr")


In [8]:
raw_neighbourhoods_table[1:]
;

''

In [9]:
len(raw_neighbourhoods_table[1:])

140

In [142]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate are {}, {}.'.format(latitude, longitude))

The geograpical coordinate are 43.6534817, -79.3839347.


In [182]:
Toronto_Neighbourhoods = pd.DataFrame(columns=["Borough","Neighbourhood","Latitude","Longitude"])
geolocator = Nominatim(user_agent="ny_explorer")

for row in raw_neighbourhoods_table[1:]:
    col = row.find_all("td")
    #CDN = col[0].get_text().replace("\n","")
    Neighbourhood = col[1].get_text().replace("\n","")
    Borough = col[2].get_text().replace("\n","")
    #Subneighbourhoods = col[3].get_text()
    #Map = col[4].get_text()
    
    if Neighbourhood is not None: 
        address = Neighbourhood+','+'Toronto'
        location = geolocator.geocode(address)
        if(location is not None):
            latitude = location.latitude
            longitude = location.longitude
            Toronto_Neighbourhoods = Toronto_Neighbourhoods.append({"Borough":Borough,"Neighbourhood":Neighbourhood, "Latitude":latitude,"Longitude":longitude}, ignore_index=True)
        else:
            Toronto_Neighbourhoods = Toronto_Neighbourhoods.append({"Borough":Borough,"Neighbourhood":Neighbourhood}, ignore_index=True)

In [183]:
Toronto_Neighbourhoods.head()


,Borough,Neighbourhood,Latitude,Longitude
0,Scarborough,Agincourt North,43.808038,-79.266439
1,Scarborough,Agincourt South-Malvern West,43.781969,-79.257689
2,Etobicoke,Alderwood,43.601717,-79.545232
3,Old City of Toronto,Annex,43.670338,-79.407117
4,North York,Banbury-Don Mills,43.752683,-79.365270


In [184]:
Toronto_Neighbourhoods.shape

(140, 4)

In [197]:
Toronto_Neighbourhoods_LL = Toronto_Neighbourhoods[~Toronto_Neighbourhoods['Latitude'].isna()]

In [198]:
Toronto_Neighbourhoods_LL.shape

(112, 4)

In [194]:
Toronto_Neighbourhoods['Neighbourhood'][134]
address = Neighbourhood+','+'Toronto'
location = geolocator.geocode(address)
print(location.longitude)

-79.45291835752212


In [199]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(Toronto_Neighbourhoods_LL['Latitude'], Toronto_Neighbourhoods_LL['Longitude'], Toronto_Neighbourhoods_LL['Borough'], Toronto_Neighbourhoods_LL['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [201]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(Toronto_Neighbourhoods_LL['Borough'].unique()),
        Toronto_Neighbourhoods_LL.shape[0]
    )
)

The dataframe has 6 boroughs and 112 neighborhoods.


In [202]:
toronto_data = Toronto_Neighbourhoods_LL
toronto_data.head()

,Borough,Neighbourhood,Latitude,Longitude
0,Scarborough,Agincourt North,43.808038,-79.266439
1,Scarborough,Agincourt South-Malvern West,43.781969,-79.257689
2,Etobicoke,Alderwood,43.601717,-79.545232
3,Old City of Toronto,Annex,43.670338,-79.407117
4,North York,Banbury-Don Mills,43.752683,-79.365270


### In this section we will start to use the Foursquare API to collect the data to be able to segment the available neighbourhoos in the toronto_data dataframe

In [1]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
ACCESS_TOKEN = '' # your FourSquare Access Token
print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: 
CLIENT_SECRET:


In [205]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [206]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [207]:
nearby_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],latitudes=toronto_data['Latitude'],longitudes=toronto_data['Longitude'])

Agincourt North
Agincourt South-Malvern West
Alderwood
Annex
Banbury-Don Mills
Bathurst Manor
Bay Street Corridor
Bayview Village
Bayview Woods-Steeles
Bendale
Black Creek
Broadview North
Cabbagetown-South St. James Town
Casa Loma
Centennial Scarborough
Church-Yonge Corridor
Clanton Park
Cliffcrest
Corso Italia-Davenport
Crescent Town
Danforth - East York
Danforth Village - Toronto
Don Valley Village
Dorset Park
Dovercourt-Wallace Emerson-Junction
Dufferin Grove
East End-Danforth
Edenbridge-Humber Valley
Eglinton East
Elms-Old Rexdale
Englemount-Lawrence
Etobicoke West Mall
Flemingdon Park
Forest Hill North
Forest Hill South
Glenfield-Jane Heights
Guildwood
Henry Farm
High Park North
High Park-Swansea
Highland Creek
Hillcrest Village
Humber Heights-Westmount
Humber Summit
Humewood-Cedarvale
Ionview
Islington-City Centre West
Junction Area
Keelesdale-Eglinton West
Kennedy Park
Kingsway South
L'Amoreaux
Lansing-Westgate
Lawrence Park North
Lawrence Park South
Leaside-Bennington
Little Po

In [208]:
toronto_venues=nearby_venues
print(toronto_venues.shape)
toronto_venues.head()

(2453, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Agincourt North,43.808038,-79.266439,Saravanaa Bhavan South Indian Restaurant,43.810117,-79.269275,Indian Restaurant
1,Agincourt North,43.808038,-79.266439,Menchie's,43.808338,-79.268288,Frozen Yogurt Shop
2,Agincourt North,43.808038,-79.266439,Booster Juice,43.809915,-79.269382,Juice Bar
3,Agincourt North,43.808038,-79.266439,Dollarama,43.808894,-79.269854,Discount Store
4,Agincourt North,43.808038,-79.266439,Congee Town 太皇名粥,43.809035,-79.267634,Chinese Restaurant


In [210]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt North,27,27,27,27,27,27
Agincourt South-Malvern West,14,14,14,14,14,14
Alderwood,6,6,6,6,6,6
Annex,40,40,40,40,40,40
Banbury-Don Mills,4,4,4,4,4,4
...,...,...,...,...,...,...
Wychwood,54,54,54,54,54,54
Yonge and Eglinton,69,69,69,69,69,69
Yonge-St.Clair,56,56,56,56,56,56


In [211]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 262 uniques categories.


In [217]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,ATM,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Argentinian Restaurant,Art Gallery,Art Museum,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt North,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Agincourt North,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Agincourt North,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Agincourt North,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Agincourt North,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [218]:
toronto_onehot.shape

(2453, 263)

In [219]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,ATM,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Argentinian Restaurant,Art Gallery,Art Museum,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt North,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.037037,0.0,0.0,0.000000,0.000,0.0,0.000000
1,Agincourt South-Malvern West,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000,0.0,0.000000
2,Alderwood,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000,0.0,0.000000
3,Annex,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.025000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.025,0.0,0.000000
4,Banbury-Don Mills,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,Wychwood,0.0,0.0,0.0,0.0,0.018519,0.0,0.000000,0.018519,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000,0.0,0.000000
108,Yonge and Eglinton,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.014493,0.014493,0.0,0.014493,0.0,0.0,0.014493,0.000,0.0,0.028986
109,Yonge-St.Clair,0.0,0.0,0.0,0.0,0.017857,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.017857,0.0,0.0,0.000000,0.000,0.0,0.017857
110,York University Heights,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.052632,0.0,0.0,0.000000,0.000,0.0,0.000000


In [220]:
toronto_grouped.shape

(112, 263)

In [221]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt North----
                venue  freq
0                Bank  0.07
1              Bakery  0.07
2  Chinese Restaurant  0.07
3          Beer Store  0.04
4         Men's Store  0.04


----Agincourt South-Malvern West----
                  venue  freq
0            Restaurant  0.29
1           Salad Place  0.07
2  Gym / Fitness Center  0.07
3        Breakfast Spot  0.07
4             Bookstore  0.07


----Alderwood----
            venue  freq
0     Pizza Place  0.33
1  Sandwich Place  0.17
2             Gym  0.17
3     Coffee Shop  0.17
4             Pub  0.17


----Annex----
             venue  freq
0      Pizza Place  0.10
1      Coffee Shop  0.08
2    Grocery Store  0.08
3           Bistro  0.05
4  Thai Restaurant  0.05


----Banbury-Don Mills----
                 venue  freq
0          Gas Station  0.25
1         Intersection  0.25
2  Japanese Restaurant  0.25
3                 Park  0.25
4                  ATM  0.00


----Bathurst Manor----
               venue  freq
0    

                     venue  freq
0     Fast Food Restaurant   0.4
1       Chinese Restaurant   0.2
2            Grocery Store   0.2
3         Asian Restaurant   0.2
4  New American Restaurant   0.0


----Kingsway South----
                venue  freq
0         Coffee Shop  0.09
1  Italian Restaurant  0.06
2                 Pub  0.06
3    Sushi Restaurant  0.06
4      Breakfast Spot  0.06


----L'Amoreaux----
                  venue  freq
0    Chinese Restaurant  0.25
1     Electronics Store  0.12
2         Shopping Mall  0.12
3  Caribbean Restaurant  0.12
4           Coffee Shop  0.12


----Lansing-Westgate----
                     venue  freq
0  Health & Beauty Service   0.5
1                      Spa   0.5
2                      ATM   0.0
3             Noodle House   0.0
4               Non-Profit   0.0


----Lawrence Park North----
                  venue  freq
0      Sushi Restaurant  0.08
1    Italian Restaurant  0.08
2                Bakery  0.06
3           Coffee Shop  0.06
4  

                venue  freq
0         Coffee Shop  0.22
1          Restaurant  0.09
2         Bus Station  0.09
3        Burger Joint  0.04
4  Mexican Restaurant  0.04


----West Hill----
                  venue  freq
0           Pizza Place  0.09
1                  Bank  0.06
2  Fast Food Restaurant  0.06
3            Restaurant  0.06
4        Breakfast Spot  0.06


----West Humber-Clairville----
               venue  freq
0               Bank  0.08
1   Video Game Store  0.08
2         Theme Park  0.08
3  Food & Drink Shop  0.08
4         Restaurant  0.08


----Weston----
                       venue  freq
0              Train Station  0.11
1                   Pharmacy  0.06
2                Coffee Shop  0.06
3               Soccer Field  0.06
4  Middle Eastern Restaurant  0.06


----Willowdale East----
                 venue  freq
0          Coffee Shop  0.15
1  Japanese Restaurant  0.06
2        Grocery Store  0.06
3           Restaurant  0.04
4                  Gym  0.04


----Will

In [222]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [223]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt North,Bank,Bakery,Chinese Restaurant,Beer Store,Men's Store,Discount Store,Sporting Goods Shop,Frozen Yogurt Shop,Fried Chicken Joint,Movie Theater
1,Agincourt South-Malvern West,Restaurant,Salad Place,Gym / Fitness Center,Breakfast Spot,Bookstore,Electronics Store,Coffee Shop,Sporting Goods Shop,Department Store,Japanese Restaurant
2,Alderwood,Pizza Place,Sandwich Place,Gym,Coffee Shop,Pub,Museum,Music Venue,Nail Salon,Neighborhood,Moving Target
3,Annex,Pizza Place,Coffee Shop,Grocery Store,Bistro,Thai Restaurant,Middle Eastern Restaurant,Pharmacy,Sandwich Place,Sushi Restaurant,Pub
4,Banbury-Don Mills,Gas Station,Intersection,Japanese Restaurant,Park,ATM,Nail Salon,Non-Profit,Nightclub,New American Restaurant,Neighborhood


In [225]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 1, 1, 2, 2, 4, 2])

In [226]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,Agincourt North,43.808038,-79.266439,2,Bank,Bakery,Chinese Restaurant,Beer Store,Men's Store,Discount Store,Sporting Goods Shop,Frozen Yogurt Shop,Fried Chicken Joint,Movie Theater
1,Scarborough,Agincourt South-Malvern West,43.781969,-79.257689,2,Restaurant,Salad Place,Gym / Fitness Center,Breakfast Spot,Bookstore,Electronics Store,Coffee Shop,Sporting Goods Shop,Department Store,Japanese Restaurant
2,Etobicoke,Alderwood,43.601717,-79.545232,2,Pizza Place,Sandwich Place,Gym,Coffee Shop,Pub,Museum,Music Venue,Nail Salon,Neighborhood,Moving Target
3,Old City of Toronto,Annex,43.670338,-79.407117,2,Pizza Place,Coffee Shop,Grocery Store,Bistro,Thai Restaurant,Middle Eastern Restaurant,Pharmacy,Sandwich Place,Sushi Restaurant,Pub
4,North York,Banbury-Don Mills,43.752683,-79.365270,1,Gas Station,Intersection,Japanese Restaurant,Park,ATM,Nail Salon,Non-Profit,Nightclub,New American Restaurant,Neighborhood


In [227]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### cluster 1

In [229]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
26,Cliffcrest,Ice Cream Shop,Restaurant,Auto Garage,Pizza Place,Pharmacy,Pet Store,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Movie Theater
38,Eglinton East,Ice Cream Shop,Indian Restaurant,Restaurant,Train Station,Sandwich Place,ATM,Neighborhood,Noodle House,Non-Profit,Nightclub


### cluster 2

In [230]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Banbury-Don Mills,Gas Station,Intersection,Japanese Restaurant,Park,ATM,Nail Salon,Non-Profit,Nightclub,New American Restaurant,Neighborhood
5,Bathurst Manor,Playground,Convenience Store,Baseball Field,Park,Neighborhood,Noodle House,Non-Profit,Nightclub,New American Restaurant,Nail Salon
28,Crescent Town,Convenience Store,Park,Metro Station,Golf Course,Noodle House,Non-Profit,Nightclub,New American Restaurant,Neighborhood,Nail Salon
37,Edenbridge-Humber Valley,Park,Dog Run,Garden,ATM,Non-Profit,Nightclub,New American Restaurant,Neighborhood,Nail Salon,Office
40,Englemount-Lawrence,Park,Bakery,Shoe Store,Shopping Mall,Coffee Shop,Noodle House,Non-Profit,Nightclub,New American Restaurant,ATM
44,Forest Hill North,Arts & Crafts Store,Playground,Park,Bank,Nail Salon,Noodle House,Non-Profit,Nightclub,New American Restaurant,Neighborhood
45,Forest Hill South,Arts & Crafts Store,Playground,Park,Bank,Nail Salon,Noodle House,Non-Profit,Nightclub,New American Restaurant,Neighborhood
49,Henry Farm,Park,Lawyer,Tennis Court,Intersection,ATM,Non-Profit,Nightclub,New American Restaurant,Neighborhood,Nail Salon
50,High Park North,Park,Gym / Fitness Center,Convenience Store,Baseball Field,Tennis Court,Mattress Store,ATM,Non-Profit,Nightclub,New American Restaurant
51,High Park-Swansea,Park,Gym / Fitness Center,Light Rail Station,Beach,Building,American Restaurant,Bus Stop,Lake,Trail,Gas Station


### cluster 3

In [231]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt North,Bank,Bakery,Chinese Restaurant,Beer Store,Men's Store,Discount Store,Sporting Goods Shop,Frozen Yogurt Shop,Fried Chicken Joint,Movie Theater
1,Agincourt South-Malvern West,Restaurant,Salad Place,Gym / Fitness Center,Breakfast Spot,Bookstore,Electronics Store,Coffee Shop,Sporting Goods Shop,Department Store,Japanese Restaurant
2,Alderwood,Pizza Place,Sandwich Place,Gym,Coffee Shop,Pub,Museum,Music Venue,Nail Salon,Neighborhood,Moving Target
3,Annex,Pizza Place,Coffee Shop,Grocery Store,Bistro,Thai Restaurant,Middle Eastern Restaurant,Pharmacy,Sandwich Place,Sushi Restaurant,Pub
6,Bay Street Corridor,Boutique,Coffee Shop,French Restaurant,Hotel,Restaurant,Italian Restaurant,Japanese Restaurant,Café,Clothing Store,Sushi Restaurant
...,...,...,...,...,...,...,...,...,...,...,...
135,Wychwood,Restaurant,Italian Restaurant,Ice Cream Shop,Coffee Shop,Bakery,Indian Restaurant,Sushi Restaurant,Café,Burger Joint,Pizza Place
136,Yonge and Eglinton,Coffee Shop,Italian Restaurant,Restaurant,Pub,Yoga Studio,Middle Eastern Restaurant,Bookstore,Movie Theater,Bakery,Gym
137,Yonge-St.Clair,Coffee Shop,Italian Restaurant,Thai Restaurant,Café,Sushi Restaurant,Grocery Store,Bank,Pharmacy,Pizza Place,Gym
138,York University Heights,Fast Food Restaurant,Pizza Place,Grocery Store,Discount Store,Fried Chicken Joint,Beer Store,Falafel Restaurant,Caribbean Restaurant,Gym / Fitness Center,Coffee Shop


### cluster 4

In [232]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Centennial Scarborough,Park,ATM,Office,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Moving Target,Museum,Music Venue
55,Humber Summit,Park,Gift Shop,Bakery,Noodle House,Non-Profit,Nightclub,New American Restaurant,Neighborhood,Nail Salon,ATM


### cluster 5

In [233]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Bayview Woods-Steeles,Dog Run,ATM,Opera House,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon
